In [1]:
# for each basin, plot basin bounds on the mosart domain subset

In [29]:
import pandas as pd
import numpy as np
import os, sys, glob
import matplotlib.pyplot as plt
import xarray as xr
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.colors import LogNorm

In [4]:
file = '/glade/work/guoqiang/CTSM_CAMELS/data_mesh_surf/HillslopeHydrology/CAMELS_level1_basin_info.csv'
df_info = pd.read_csv(file)

In [25]:
# # plot basin location

# for basin in range(len(df_info)):
#     file = f'/glade/work/guoqiang/CTSM_CAMELS/data_mosart/CAMELS_level1_{basin}_MOSART_routing_0.125nldas2.nc'
#     ds_mosart = xr.open_dataset(file)

#     # plot

#     # plot mosart total area
#     ds_mosart['lon'] = ds_mosart.longxy.values[0,:]
#     ds_mosart['areaTotal'] = ds_mosart['areaTotal'].where(ds_mosart['dnID'].values >= 0, np.nan)

#     # plot basin
#     latb = df_info.iloc[basin]['latoutlet']
#     lonb = df_info.iloc[basin]['lonoutlet']
    
#     file_mesh = f'/glade/work/guoqiang/CTSM_CAMELS/data_mesh_surf/HillslopeHydrology/disaggregation/corrected_HCDN_nhru_final_671_buff_fix_holes.CAMELSandTDX_areabias_fix.simp0.001.level1_polygons_neighbor_group_esmf_mesh_{basin}.nc'
#     ds_basin = xr.load_dataset(file_mesh)
#     nodeCoords = ds_basin.nodeCoords.values
    
#     ds_mosart.areaTotal.plot()
#     plt.scatter(nodeCoords[:,0]-360, nodeCoords[:,1], 10, color='pink')
#     plt.scatter(lonb, latb, 50, color='r')
#     plt.show()

#     break

In [40]:
pdf_filename = "CAMELS_level1_MOSARTdomain.pdf"
with PdfPages(pdf_filename) as pdf:
    for basin in range(len(df_info)):
        file = f'/glade/work/guoqiang/CTSM_CAMELS/data_mosart/CAMELS_level1_{basin}_MOSART_routing_0.125nldas2.nc'
        ds_mosart = xr.open_dataset(file)

        # Plot MOSART total area
        ds_mosart['lon'] = ds_mosart.longxy.values[0, :]
        ds_mosart['areaTotal'] = ds_mosart['areaTotal'].where(ds_mosart['dnID'].values >= 0, np.nan)

        # Get basin coordinates
        latb = df_info.iloc[basin]['latoutlet']
        lonb = df_info.iloc[basin]['lonoutlet']
        
        # Load basin mesh data
        file_mesh = f'/glade/work/guoqiang/CTSM_CAMELS/data_mesh_surf/HillslopeHydrology/disaggregation/corrected_HCDN_nhru_final_671_buff_fix_holes.CAMELSandTDX_areabias_fix.simp0.001.level1_polygons_neighbor_group_esmf_mesh_{basin}.nc'
        ds_basin = xr.load_dataset(file_mesh)
        nodeCoords = ds_basin.nodeCoords.values
        
        # Plotting
        plt.figure()
        da = ds_mosart.areaTotal/1e6 # m2 to km2
        usgsarea = df_info.iloc[basin]['areaUSGS']
        da.plot(vmin=0, vmax=usgsarea*1.5, cmap='jet')
        plt.scatter(nodeCoords[:, 0] - 360, nodeCoords[:, 1], 10, color='pink')
        plt.scatter(lonb, latb, 50, color='k')
        plt.title(f"Basin {basin} - Area Total - USGSarea {usgsarea}")
        
        # Save the current figure to the PDF
        pdf.savefig()  # saves the current figure into a pdf page
        plt.close()  # close the current figure